In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv("../.env.local")

from polybridge import PolybridgeClient
from datetime import datetime, timedelta, timezone

import pandas as pd

In [ ]:
client = PolybridgeClient(api_key=os.getenv("POLYBRIDGE_API_KEY"))

In [ ]:
# Test the new endpoint
end_time = datetime.now(timezone.utc)
start_time = end_time - timedelta(hours=8)

result = client.fetch_up_or_down_options_timeseries(
    asset="BTC",
    start_ts=start_time.isoformat(),
    end_ts=end_time.isoformat(),
    window_days=[7],  # Only 7-day windows currently supported
    horizon="daily"  # Optional, defaults to "daily"
)

# Convert to DataFrame
rows = result["rows"]
df = pd.DataFrame(rows)

In [ ]:
# Check the structure
print(f"Got {len(df)} rows")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nFirst few rows:")
print(df.head())

# Check data types
print(f"\nData types:")
print(df.dtypes)

# Check for missing data
print(f"\nMissing values per column:")
print(df.isnull().sum())

In [ ]:
df[['timestamp_bucket', 'prob_up_next', 'spot_price', 'iv_next_7d', 'rv_7d']]

In [ ]:
import matplotlib.pyplot as plt

# Plot the probability of the up-or-down market
fig, ax = plt.subplots(figsize=(10, 8), nrows=3, ncols=1, sharex=True)
axx = ax[0]
axx.plot(pd.to_datetime(df['timestamp_bucket']), df['prob_up_next'], label='Probability of Up-or-Down Market')
axx2 = axx.twinx()
axx2.plot(pd.to_datetime(df['timestamp_bucket']), df['spot_price'], label='Binance Spot Price', color='orange')
axx.set_xlabel('Timestamp')
axx.set_ylabel('Probability')
# axx.legend()
axx2.set_ylabel('Spot Price')
# axx2.legend()

axx = ax[1]
axx.plot(pd.to_datetime(df['timestamp_bucket']), df['iv_next_7d'], label='Implied Volatility', color='orange')
axx.set_xlabel('Timestamp')
axx.set_ylabel('Implied Volatility')
# axx.legend()

axx = ax[2]
axx.plot(pd.to_datetime(df['timestamp_bucket']), df['rv_7d'], label='Realized Volatility', color='green')
axx.set_xlabel('Timestamp')
axx.set_ylabel('Realized Volatility')
# axx.legend()


plt.show()

